
# Data and Image Transformation in Pytorch


    Transforms are common image transformations

    from torchvision import transforms
    import torchvision.transforms.functional as F 
    
    from torchvision.transforms import v2
    import torchvision.transforms.v2.functional as F


link: https://pytorch.org/vision/stable/transforms.html


In [ ]:
#  PIL: Python Imaging Library
from PIL import Image
import torch
from torchvision import transforms
from torchvision.transforms import v2
import torchvision.transforms.v2.functional as F

In [ ]:
image = Image.open('nature.jpg') 
image

In [ ]:
# Parameters:
    # p (float) – probability of the image being flipped. Default value is 0.5
transform =  transforms.RandomHorizontalFlip(p=0.5)
out = transform(image)
out

In [ ]:
# degrees : Range of degrees to select from - (min, max)
transform =  transforms.RandomRotation(degrees=(-180,180))
out = transform(image)
out

In [ ]:
# image

In [ ]:
transform = transforms.RandomCrop(size=(224, 224))
out = transform(image)
out

In [ ]:
transform = transforms.Grayscale()
out = transform(image)
out

In [ ]:
transform =  transforms.CenterCrop(size=(200,200))
out = transform(image)
out

In [ ]:
transform =  v2.Resize(size=250) # (h, w)
out = transform(image)
out



In [ ]:
output = F.adjust_brightness(image, brightness_factor=3.0) 
output

In [ ]:
output = F.adjust_contrast(image,contrast_factor=4.2) 
output

In [ ]:
output = F.adjust_hue(image,hue_factor=-0.3) 
output

# Conversion:
- ToPILImage: from tensor or ndarray to image
- ToTensor: from numpy.ndarray or PILImage to tensor

In [ ]:
# Converts a PIL Image or numpy.ndarray in the range [0, 255] 
#                to a torch.FloatTensor in the range [0.0, 1.0]

transform =  transforms.ToTensor()
tensor_out = transform(image)
tensor_out

In [ ]:
# Converts a PIL Image or numpy.ndarray in the range [0, 255] 
#                to a torch.FloatTensor in the range [0.0, 1.0]

transform =  transforms.ToPILImage()
out = transform(tensor_out)
out

In [ ]:
# Convert a tensor, ndarray, or PIL Image to Image
# In the transforms, Image instances are largely interchangeable with pure torch.Tensor

transform = v2.ToImage()
image_tensor_out = transform(image)
image_tensor_out

In [ ]:
transform =  transforms.ToPILImage()
out = transform(image_tensor_out)
out

# Compose:

In [ ]:

transform = v2.Compose([
     v2.ToImage(), # Convert to tensor, only needed if you had a PIL image
     v2.ToDtype(torch.float32, scale=True)
 ])
compose_img = transform(image)
compose_img

In [ ]:
# image

In [ ]:
transform =  transforms.ToPILImage()
out = transform(compose_img)
out

# Normalize

### Parameters:
- mean (sequence) – Sequence of means for each channel.

- std (sequence) – Sequence of standard deviations for each channel.

- inplace (bool,optional) – Bool to make this operation in-place.

In [ ]:
transform = v2.Compose([
    v2.RandomResizedCrop(size=(224, 224)),
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToImage(), # Convert to tensor, only needed if you had a PIL image
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
])
compose_img_2 = transform(image)
compose_img_2

In [ ]:
transform =  transforms.ToPILImage()
out = transform(compose_img_2)
out

# Custom Transformation:

In [ ]:
from torch.utils.data import Dataset,DataLoader
import numpy as np

In [ ]:
#adding transform to our dataset
class WineDataset(Dataset):
    
    def __init__(self, transform=None):
        data = np.loadtxt('./wine.csv',delimiter=',',dtype=np.float32,skiprows=1)
        self.n_smaples = data.shape[0]
        
         # note that we do NOT convert to tensor here
        self.x = data[:,1:] 
        self.y = data[:,[0]] 
        
        self.transform = transform

    def __getitem__(self,index):
        sample = self.x[index], self.y[index]
        
        if self.transform:
            sample = self.transform(sample)
            
        return sample
        
    def __len__(self):
        return self.n_smaples
# we have not transformed our x, y data
# our class accepts a transform argument

### The \__call__ method 
enables Python programmers to write classes where the instances behave like functions and can be called like a function

In [ ]:
# create custom transform for our dataset class

class To_Tensor:
    def __call__(self,sample):
        inputs, targets = sample
        
        transformed_inputs  = torch.from_numpy(inputs)
        transformed_targets = torch.from_numpy(targets)
        
        return transformed_inputs , transformed_targets


In [ ]:
dataset = WineDataset()
transformed_dataset = WineDataset(transform=To_Tensor())

In [ ]:
first_data = dataset[0]
first_data_transformed =  transformed_dataset[0]

In [ ]:
first_data

In [ ]:
first_data_transformed

In [ ]:
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html